# Smart Data Quality Agent - Demo Notebook

## What This Notebook Does
This notebook demonstrates the complete Smart DQ Check workflow with enhanced null analysis:

1. **Schema Indexing** - Automatically discovers and indexes database tables (must run for first time)
2. **Smart Analysis** - Uses AI to understand your query and analyze the right tables  
3. **Enhanced Report Generation** - Creates comprehensive data quality reports with detailed null analysis in multiple formats
4. **Fixed Null Analysis Issue**: Enhanced agent prompt to ensure detailed null values breakdown is always included
5. **Complete Data Coverage**: Reports now show per-column null counts and percentages
6. **Multi-Format Output**: All formats (Markdown, HTML, JSON) contain complete null analysis details

## How to Use
1. **First Time Setup**: Run cell 4 to build schema index and discover your database tables
2. **Run Analysis**: Cell 6 prompts for your query - try examples like:
   - "Generate comprehensive DQ report for prod customers table"
   - "Create data quality report for production database invoices"
3. **Get Reports**: System generates detailed reports saved to `../reports/` in 3 formats (.html, .json, .md)

## Reports being saved under reports folder shows
- **Duplicate Record Analysis** - Complete duplicate detection and counts
- **Detailed Null Values Assessment** - Per-column breakdown with counts and percentages  
- **Statistical Summaries** - Comprehensive data profiling with descriptive stats
- **Actionable Recommendations** - Data quality improvement suggestions

**Just run all cells in order and follow the prompts**

In [1]:
import os
from dotenv import load_dotenv
import json
import re
import yaml
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from src.retrieval.schema_indexer import SchemaIndexer
from src.agent.smart_planner import run_smart_dq_check
import src.reporting.report_templates
import src.data_quality.checks
import src.reporting.report_generator
import src.agent.reporting_tools
import src.reporting.report_processor
import src.reporting.report_templates
import src.reporting.report_generator
from src.reporting.report_processor import SmartDQReportProcessor

### Schema Indexing - Always rebuild from scratch to ensure fresh data

In [2]:
# Load settings.yaml to read connector types dynamically
with open('../config/settings.yaml', 'r') as f:
    settings = yaml.safe_load(f)

# Get connector types from settings.yaml
connector_types = list(settings['connectors'].keys())

print(f" BUILDING SCHEMA INDEX FROM SCRATCH")

# First connector: recreate=True
# Subsequent connectors: recreate=False to preserve other connector data
for i, ct in enumerate(connector_types):
    print(f"\n Rebuilding index for {ct.upper()}")

    # First connector recreates the entire index, others append to it
    recreate_index = (i == 0)  # Only recreate on first connector

    if recreate_index:
        print(f"  Starting fresh - recreating entire index")
    else:
        print(f"  Adding to existing index")

    indexer = SchemaIndexer(connector_type=ct)
    indexer.build_schema_index(recreate=recreate_index)
    print(f"   Index {'created' if recreate_index else 'updated'} for {ct}")

print(f"\n SCHEMA INDEX REBUILD COMPLETE!")

 BUILDING SCHEMA INDEX FROM SCRATCH

 Rebuilding index for SNOWFLAKE
  Starting fresh - recreating entire index
 Connecting to Snowflake...
 Connecting to Snowflake...
✓ Connected to Snowflake: PROD_SALES.PUBLIC
✓ Connected to Snowflake: PROD_SALES.PUBLIC
✓ Disconnected from Snowflake
Auto-discovered 4 schemas on Snowflake: DATA_MART_FINANCE, DATA_MART_MANUFACTURING, DATA_MART_MARKETING, PUBLIC
[1/4] INDEXING SCHEMA: DATA_MART_FINANCE
✓ Disconnected from Snowflake
Auto-discovered 4 schemas on Snowflake: DATA_MART_FINANCE, DATA_MART_MANUFACTURING, DATA_MART_MARKETING, PUBLIC
[1/4] INDEXING SCHEMA: DATA_MART_FINANCE

Generating metadata documents for 2 tables...
  [1/2] Processing INCOME_BAND...

Generating metadata documents for 2 tables...
  [1/2] Processing INCOME_BAND...
  [2/2] Processing INVOICES...
  [2/2] Processing INVOICES...


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


 Deleted existing collection: database_schemas

Indexing 2 table metadata documents...
Indexed 2 tables from schema DATA_MART_FINANCE
[2/4] INDEXING SCHEMA: DATA_MART_MANUFACTURING
Indexed 2 tables from schema DATA_MART_FINANCE
[2/4] INDEXING SCHEMA: DATA_MART_MANUFACTURING

Generating metadata documents for 1 tables...
  [1/1] Processing PRODUCTS...

Generating metadata documents for 1 tables...
  [1/1] Processing PRODUCTS...


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



Indexing 1 table metadata documents...
Indexed 1 tables from schema DATA_MART_MANUFACTURING
[3/4] INDEXING SCHEMA: DATA_MART_MARKETING

Generating metadata documents for 3 tables...
  [1/3] Processing CALL_CENTER...

Generating metadata documents for 3 tables...
  [1/3] Processing CALL_CENTER...
  [2/3] Processing CUSTOMERS...
  [2/3] Processing CUSTOMERS...
  [3/3] Processing SALES...
  [3/3] Processing SALES...


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



Indexing 3 table metadata documents...
Indexed 3 tables from schema DATA_MART_MARKETING
[4/4] INDEXING SCHEMA: PUBLIC

Generating metadata documents for 0 tables...
No tables found in schema PUBLIC
 SCHEMA INDEX BUILT SUCCESSFULLY
  - Collection: database_schemas
  - Schemas indexed: 4
  - Total tables indexed: 6
   Index created for snowflake

 Rebuilding index for POSTGRES
  Adding to existing index

Generating metadata documents for 0 tables...
No tables found in schema PUBLIC
 SCHEMA INDEX BUILT SUCCESSFULLY
  - Collection: database_schemas
  - Schemas indexed: 4
  - Total tables indexed: 6
   Index created for snowflake

 Rebuilding index for POSTGRES
  Adding to existing index
 Connecting to postgres...
 Connecting to postgres...
✓ Connected to PostgreSQL: stage_sales
✓ Disconnected from PostgreSQL
Auto-discovered 1 schemas on postgres: public
[1/1] INDEXING SCHEMA: public
Discovering tables in stage_sales.public...
✓ Found 3 tables/views

Generating metadata documents for 3 tab

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



Indexing 3 table metadata documents...
Indexed 3 tables from schema public
 SCHEMA INDEX BUILT SUCCESSFULLY
  - Collection: database_schemas
  - Schemas indexed: 1
  - Total tables indexed: 3
   Index updated for postgres

 SCHEMA INDEX REBUILD COMPLETE!


## Enhanced Comprehensive Report Creation

Generate comprehensive data quality reports with detailed duplicate counts, null analysis, and descriptive statistics.

In [16]:
# Get user input for the query with a default option
default_query = "Generate comprehensive DQ report for prod customers table"
user_query = input("Enter your DQ check query (or press Enter to use default): ").strip() or default_query
print(f"Using query: {user_query}")

comprehensive_report = run_smart_dq_check(user_query)

Using query: comprehensive report staging database customers table

SMART DATA QUALITY CHECK
User Query: comprehensive report staging database customers table

Step 1: Discovering relevant tables...
----------------------------------------------------------------------


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Number of requested results 20 is greater than number of elements in index 9, updating n_results = 9
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Number of requested results 20 is greater than number of elements in index 9, updating n_results = 9
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given



Found 1 relevant table(s) (≥5% relevance):

  1. [POSTGRES] stage_sales.public.customers
     Relevance: 5.17% ✓

Step 2: Building context for agent...
----------------------------------------------------------------------

Step 3: Running agent with discovered tables...
----------------------------------------------------------------------


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `generate_comprehensive_dq_report` with `{'dataset_id': 'stage_sales.public.customers', 'connector_type': 'postgres', 'output_format': 'markdown'}`


Running duplicates check...
--- Loading data for: stage_sales.public.customers using POSTGRES connector ---
✓ Connected to PostgreSQL: stage_sales
Executing query: SELECT * FROM stage_sales.public.customers

Invoking: `generate_comprehensive_dq_report` with `{'dataset_id': 'stage_sales.public.customers', 'connector_type': 'postgres', 'output_format': 'markdown'}`


Running duplicates check...
--- Loading data fo

In [17]:
# Initialize the report processor
processor = SmartDQReportProcessor(reports_dir="../reports")

# Process the comprehensive report (handles validation, extraction, checks, and file generation)
result = processor.process_comprehensive_report(comprehensive_report)

Smart DQ check successful - proceeding with report generation...
Extracted info:
   Dataset: stage_sales.public.customers
   Connector: postgres

Running comprehensive DQ assessment with reliable data sources...
   Executing duplicates check...
--- Loading data for: stage_sales.public.customers using POSTGRES connector ---
✓ Connected to PostgreSQL: stage_sales
Executing query: SELECT * FROM stage_sales.public.customers
✓ Loaded 50000 rows from PostgreSQL
✓ Disconnected from PostgreSQL
   Executing null_values check...
--- Loading data for: stage_sales.public.customers using POSTGRES connector ---
✓ Loaded 50000 rows from PostgreSQL
✓ Disconnected from PostgreSQL
   Executing null_values check...
--- Loading data for: stage_sales.public.customers using POSTGRES connector ---
✓ Connected to PostgreSQL: stage_sales
Executing query: SELECT * FROM stage_sales.public.customers
✓ Connected to PostgreSQL: stage_sales
Executing query: SELECT * FROM stage_sales.public.customers
✓ Loaded 50000 r